In [2]:
import keras
import numpy as np
from PIL import Image
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D
# from misc import load_data1, load_data2, load_data3, load_data4, load_data5, load_data6, load_data7, load_data8
from keras_vggface.vggface import VGGFace
from keras.models import Model, Sequential
from keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D

In [3]:
# data = load_data1()

K.set_learning_phase(1)

sess = K.get_session()
X = K.tf.placeholder(shape=(None, 1818, 1228, 3), dtype=K.tf.float32)
Y = K.tf.placeholder(shape=(None, 5), dtype=K.tf.float32)

In [4]:
vgg = VGGFace(include_top=False, input_tensor = X, input_shape=(1228, 1818, 3))
for layer in vgg.layers:
    layer.trainable = False

In [5]:
vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1228, 1818, 3)     0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 1228, 1818, 64)    1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 1228, 1818, 64)    36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 614, 909, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 614, 909, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 614, 909, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 307, 454, 128)     0         
__________

In [168]:
p1 = Conv2D(filters=10, kernel_size=3, strides=1, padding='same')(vgg.get_layer(index=13).output)
p2 = Conv2D(filters=10, kernel_size=3, strides=1, padding='same')(vgg.get_layer(index=17).output)
p3 = Conv2D(filters=10, kernel_size=5, strides=1, padding='same')(MaxPool2D(strides=2)(p2))
p4 = Conv2D(filters=10, kernel_size=5, strides=1, padding='same')(MaxPool2D(strides=2)(p3))

p11 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p1))
p12 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p1)
p21 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p2))
p22 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p2)
p31 = K.sigmoid(Conv2D(filters=1, kernel_size=5, strides=1, padding='same')(p3))
p32 = Conv2D(filters=4, kernel_size=5, strides=1, padding='same')(p3)
p41 = K.sigmoid(Conv2D(filters=1, kernel_size=5, strides=1, padding='same')(p4))
p42 = Conv2D(filters=4, kernel_size=5, strides=1, padding='same')(p4)

# p12 = Conv2D(filters=5, kernel_size=3, strides=1, padding='same')(p2)
# p22 = Conv2D(filters=5, kernel_size=3, strides=1, padding='same')(p2)
# p32 = Conv2D(filters=5, kernel_size=5, strides=1, padding='same')(p3)
# p42 = Conv2D(filters=5, kernel_size=5, strides=1, padding='same')(p4)

p12 = K.concatenate(tensors=[p11, p12], axis = -1)
p22 = K.concatenate(tensors=[p21, p22], axis = -1)
p32 = K.concatenate(tensors=[p31, p32], axis = -1)
p42 = K.concatenate(tensors=[p41, p42], axis = -1)

In [170]:
pred = ([K.reshape(p12, (-1, 5)), K.reshape(p22, (-1, 5)), K.reshape(p32, (-1, 5)), K.reshape(p42, (-1, 5))])

In [190]:
def NMS(boxes = pred):
    
#     Z0 = K.tf.image.non_max_suppression(boxes=K.reshape(boxes[0][0][:, 1:], (-1, 4)), scores=K.reshape(boxes[0][0][:, 0], (-1, )),
#                                         max_output_size=25, iou_threshold=0.55)
#     Z1 = K.tf.image.non_max_suppression(boxes=K.reshape(boxes[0][1][:, 1:], (-1, 4)), scores=K.reshape(boxes[0][1][:, 0], (-1, )),
#                                         max_output_size=25, iou_threshold=0.55)
#     Z2 = K.tf.image.non_max_suppression(boxes=K.reshape(boxes[0][2][:, 1:], (-1, 4)), scores=K.reshape(boxes[0][2][:, 0], (-1, )),
#                                         max_output_size=25, iou_threshold=0.55)
#     Z3 = K.tf.image.non_max_suppression(boxes=K.reshape(boxes[0][3][:, 1:], (-1, 4)), scores=K.reshape(boxes[0][3][:, 0], (-1, )),
#                                         max_output_size=25, iou_threshold=0.55)
    
#     A = K.reshape(K.concatenate(tensors= [K.gather(boxes[0][0], Z0), K.gather(boxes[0][1], Z1), K.gather(boxes[0][2], Z2), K.gather(boxes[0][3], Z3)], axis = 0), (-1, 5))

    Z0 = K.tf.image.non_max_suppression(boxes=K.reshape(boxes[0][:, 1:], (-1, 4)), scores=K.reshape(boxes[0][:, 0], (-1, )),
                                        max_output_size=25, iou_threshold=0.55)
    Z1 = K.tf.image.non_max_suppression(boxes=K.reshape(boxes[1][:, 1:], (-1, 4)), scores=K.reshape(boxes[1][:, 0], (-1, )),
                                        max_output_size=25, iou_threshold=0.55)
    Z2 = K.tf.image.non_max_suppression(boxes=K.reshape(boxes[2][:, 1:], (-1, 4)), scores=K.reshape(boxes[2][:, 0], (-1, )),
                                        max_output_size=25, iou_threshold=0.55)
    Z3 = K.tf.image.non_max_suppression(boxes=K.reshape(boxes[3][:, 1:], (-1, 4)), scores=K.reshape(boxes[3][:, 0], (-1, )),
                                        max_output_size=25, iou_threshold=0.55)
    
    A = K.reshape(K.concatenate(tensors= [K.gather(boxes[0], Z0), K.gather(boxes[1], Z1), K.gather(boxes[2], Z2), K.gather(boxes[3], Z3)], axis = 0), (-1, 5))
    
    return A

In [172]:
def NMS_2(A):
    Z0 = K.tf.image.non_max_suppression(boxes = K.reshape(A[0][:, 1:] , (-1, 4)), scores= K.reshape(A[0][:, 0], (-1, )),
                                        max_output_size = 25, iou_threshold=0.55)
    return K.gather(A[0], Z0)

In [106]:
def sigmoid(X):
    return 1/(1 + K.exp(-X))

def relu(X):
    return K.maximum(X, 0)

def loss(Y, y_, a=0.6):
    if K.tf.size(Y) != K.tf.size(y_):
        y_ = y_[:2, :]
    softMax = K.mean(-(Y[:, 0] * K.log(sigmoid(y_[:, 0])) + ((1 - Y[:, 0]) * K.log(sigmoid(1 - y_[:, 0])))))
    mse = K.mean(keras.losses.mean_squared_error(Y[:, 1:], relu(y_[:, 1:])))
    return softMax + a*mse 

In [191]:
sess.run(K.tf.global_variables_initializer())

In [192]:
b = sess.run([NMS()], feed_dict={X: np.expand_dims(np.array(Image.open("x.jpg")), axis = 0)})

In [ ]:
opt = K.tf.train.AdagradOptimizer(learning_rate=0.001)
train_ = opt.minimize(Loss)
sess.run(K.tf.global_variables_initializer())

In [10]:
def train(epochs = 10):
    for e in range(epochs):
        cost = 0
#         for i, d in enumerate(data):
        _, L = sess.run([train_, Loss], feed_dict={X: np.expand_dims(np.array(Image.open("x.jpg")), axis=0), Y: np.ones((2, 5)).reshape((-1, 5))})
        cost += L
        if e % 100 == 0:
            print("epoch {}, iter {} , loss {}".format(e, i , cost/len(data)))

In [ ]:
train(epochs = 12)